# Working with Redshift Data

### Introduction

In this notebook we illustrate how to copy data from Redshift to S3 and vice-versa.
Prerequisites

In order to successfully run this notebook, you'll first need to:
1. Have a Redshift cluster within the same VPC.
1. Preload that cluster with data from the iris data set in a table named public.irisdata.
1. Update the credential file (redshift_creds_template.json.nogit) file with the appropriate information.

Also, note that this Notebook instance needs to resolve to a private IP when connecting to the Redshift instance. There are two ways to resolve the Redshift DNS name to a private IP:
1. The Redshift cluster is not publicly accessible so by default it will resolve to private IP.
1. The Redshift cluster is publicly accessible and has an EIP associated with it but when accessed from within a VPC, it should resolve to private IP of the Redshift cluster. This is possible by setting following two VPC attributes to yes: DNS resolution and DNS hostnames. For instructions on setting that up, see Redshift public docs on Managing Clusters in an Amazon Virtual Private Cloud (VPC).


### Get Started!

1. log into redshift console

2. quick launch cluster with single ds2.large node (try dc2 first, takes about 15 mins to launch)

3. create a bucket called sagemaker-redshift-data and upload the iris.csv file

4. remember to add redshiftFullAccess (or appropriate) to the Sagemaker notebook instance role

5. Open the redshift master security group, and select an add new rule for inbound connections to the redshift_master securty group. Add the port 5439 and an appropriate inbound rule CIDR

#### Notebook Setup
Let's start by installing psycopg2, a PostgreSQL database adapter for the Python, adding a few imports and specifying a few configs.

In [ ]:
!conda install -y -c anaconda psycopg2

In [ ]:
import os
import boto3
import pandas as pd
import json
import psycopg2
import sqlalchemy as sa

region = boto3.Session().region_name

bucket='sagemaker-redshift-data' # put your s3 bucket name here, and create s3 bucket
prefix = ''
# customize to your bucket where you have stored the data

credfile = 'redshift_creds_template.json.nogit'

### Some Base functions

In [ ]:
def get_conn(creds): 
    conn = psycopg2.connect(dbname=creds['db_name'], 
                            user=creds['username'], 
                            password=creds['password'],
                            port=creds['port_num'],
                            host=creds['host_name'])
    return conn

def get_df(creds, query):
    with get_conn(creds) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            result_set = cur.fetchall()
            colnames = [desc.name for desc in cur.description]
            df = pd.DataFrame.from_records(result_set, columns=colnames)
    return df

## Reading from Redshift

We store the information needed to connect to Redshift in a credentials file. See the file `redshift_creds_template.json.nogit` for an example. 

In [ ]:
!cat redshift_creds_template.json.nogit

In [ ]:
# Read credentials to a dictionary
with open(credfile) as fh:
    creds = json.loads(fh.read())

## Upload to S3

In [ ]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data --output ./iris2.csv

In [ ]:
localFile = 'iris2.csv'

In [ ]:
print("Writing to S3...")

fObj = open(localFile, 'rb')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, localFile)).upload_fileobj(fObj)
print("Done")

## Writing data to Redshift

In [ ]:
print("Reading from S3...")
# key unchanged for demo purposes - change key to read from output data
key = os.path.join(prefix, localFile)

s3 = boto3.resource('s3')
outfile = 'iris2.csv'
s3.Bucket(bucket).download_file(key, outfile)
df2 = pd.read_csv(outfile)
print("Done")

In [ ]:
print("Writing to Redshift...")

connection_str = 'postgresql+psycopg2://' + \
                  creds['username'] + ':' + \
                  creds['password'] + '@' + \
                  creds['host_name'] + ':' + \
                  creds['port_num'] + '/' + \
                  creds['db_name'];
                    
df2.to_sql('irisdata_v2', connection_str, schema='public', index=False)
print("Done")

We read the copied data in Redshift - success!

### Read data using pandas read_sql_query

In [ ]:
pd.options.display.max_rows = 10
conn = get_conn(creds)
query = 'select * from irisdata_v2'
df = pd.read_sql_query(query, conn)
df.head()

### Read data using our custom wrapper

In [ ]:
print("Reading from Redshift...")
df = get_df(creds, query)
df.head()

### Save results of "inference" to another file

In [ ]:
print("Saving file")
localFile = 'iris_read.csv'
df.to_csv(localFile, index=False)

print("Done")